<a href="https://colab.research.google.com/github/lmcatarino/Projeto_ImersaoIA/blob/main/Identificador_de_Emo%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
"""
At the command line, only need to run once to install the package via pip:

$ pip install google-generativeai
"""

from pathlib import Path
import hashlib
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

# Set up the model
generation_config = {
  "temperature": 0,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

prompt_parts = [
  "Identifique a emoção presente na imagem e faça um comentário sobre como lidar com esta emoção no dia a dia",
  "input: ​",
  "output: A alegria é uma emoção positiva caracterizada por uma sensação de felicidade, contentamento e bem-estar. Geralmente acompanhada de sorrisos, risadas e um senso de leveza, ela surge de diversas fontes, como alcançar um objetivo, passar tempo com pessoas queridas ou simplesmente apreciar um belo dia. É contagiante e pode melhorar o humor e a saúde física e mental. Cultivar a alegria no dia a dia através de atividades prazerosas, gratidão e atos de bondade contribui para uma vida mais plena e significativa.",
  "input: ​",
  "output: O medo é uma emoção humana natural e crucial para a sobrevivência, pois nos alerta para perigos reais ou imaginários. Caracteriza-se por uma apreensão desconfortável e prepara o corpo para lutar ou fugir da ameaça percebida. Fisiologicamente, o medo desencadeia aumento da frequência cardíaca, respiração acelerada, sudorese e tensão muscular. Psicologicamente, pode se manifestar desde um leve nervosismo até o terror paralisante. A intensidade varia de acordo com a severidade da ameaça, as características individuais e as experiências passadas. Compreender e gerenciar o medo é fundamental para evitar que ele controle nossas vidas.",
  "input: ​",
  "output: O nojo é uma emoção universal e instintiva que nos afasta de substâncias e situações consideradas sujas, perigosas ou moralmente repugnantes. Ele atua como um mecanismo de proteção, prevenindo doenças e nos afastando de perigos potenciais. Fisiologicamente, o nojo se manifesta através de náuseas, ânsias, salivação excessiva, expressões faciais características e até mesmo vômito. Psicologicamente, está associado a sentimentos de repulsa, desgosto e aversão. O nojo pode ser aprendido e influenciado por fatores culturais, sociais e individuais. Compreendê-lo nos ajuda a navegar no mundo e a tomar decisões que promovam nossa saúde e bem-estar.",
  "input: ​",
  "output: A raiva é uma emoção humana natural que surge em resposta a situações percebidas como injustas, frustrantes ou ameaçadoras. Caracteriza-se por um sentimento intenso de irritação, hostilidade e ressentimento, podendo levar a explosões verbais ou físicas. Fisiologicamente, a raiva dispara o sistema nervoso autônomo, aumentando a frequência cardíaca, a pressão arterial e a tensão muscular. Psicologicamente, pode se manifestar desde um leve incômodo até a fúria incontrolável. A intensidade varia de acordo com a severidade da situação, as características individuais e as experiências passadas. Compreender e gerenciar a raiva de forma saudável é crucial para evitar consequências negativas em relacionamentos e na qualidade de vida.",
  "input: ​",
  "output: O desprezo é uma emoção complexa que envolve aversão intensa e desgosto por algo ou alguém. Caracteriza-se por um sentimento de superioridade moral e um julgamento negativo do outro, percebido como inferior ou indigno de respeito. Fisiologicamente, pode gerar náuseas, sudorese e aumento da frequência cardíaca. Psicologicamente, se manifesta em expressões faciais como nariz empinado, lábios franzidos e olhar de desdém. O desprezo pode levar à exclusão social, bullying e até mesmo violência. É importante reconhecer e questionar os pensamentos e comportamentos que o motivam, buscando cultivar compaixão e empatia pelo próximo.",
  "input: ​",
  "output: A dor é uma experiência sensorial e emocional desagradável causada por lesão física ou doença. Ela pode ser aguda, de curto prazo, como uma picada de agulha, ou crônica, de longa duração, como a dor de artrite. A dor se manifesta através de sinais nociceptivos que viajam dos nervos para a medula espinhal e o cérebro, onde são interpretados como uma sensação dolorosa. A intensidade da dor varia de acordo com a causa, a tolerância individual e o estado psicológico do indivíduo. A dor pode ter um impacto significativo na qualidade de vida, afetando o humor, o sono, a capacidade de trabalhar e realizar atividades diárias. Existem diversos tipos de tratamento para a dor, como medicamentos, fisioterapia, terapia psicológica e intervenções cirúrgicas.",
  "input: ​",
  "output: A surpresa é uma emoção breve e involuntária que surge diante de eventos inesperados. Caracteriza-se por um estado de alerta fisiológico e psicológico, com aumento da frequência cardíaca, dilatação das pupilas e microexpressões faciais como arregalar os olhos e abrir a boca. Pode ser positiva, como ao receber um presente, ou negativa, como ao presenciar um acidente. A intensidade da surpresa varia de acordo com a imprevisibilidade do evento e o significado atribuído a ele.",
  "input: ​",
  "output: A tristeza é uma emoção humana comum e complexa, geralmente associada a sentimentos de desânimo, melancolia, infelicidade e desesperança. Surge como resposta natural a eventos ou circunstâncias difíceis, perdas, frustrações, decepções ou experiências emocionais negativas. A intensidade, duração e causa da tristeza variam, sendo parte normal da vida. Fisicamente, pode levar a alterações no apetite, sono, energia e imunidade. Em alguns casos, a tristeza persistente pode ser sintoma de depressão, exigindo ajuda profissional.",
  "input: ",
  genai.upload_file("/content/surpresa.jpeg"),
  "​",
  "output: ",
]

response = model.generate_content(prompt_parts)
print(response.text)

A emoção predominante na imagem parece ser a **surpresa**. O homem tem os olhos arregalados e a boca levemente aberta, expressões faciais características da surpresa. 

Lidar com a surpresa no dia a dia pode ser desafiador, pois ela surge de forma inesperada. Algumas dicas:

* **Respire fundo:** Surpresas, especialmente as negativas, podem gerar ansiedade. Respirar profundamente ajuda a acalmar o corpo e a mente.
* **Observe sua reação:** Tente entender o que te causou surpresa e como você se sente. Isso te ajuda a processar a emoção e a reagir de forma mais consciente.
* **Adapte-se à situação:** Uma vez que você entendeu a surpresa, ajuste seus planos e ações de acordo com a nova realidade.
* **Busque o lado positivo:** Mesmo em surpresas negativas, tente encontrar algo positivo ou aprender com a experiência.

Lembre-se que a surpresa é uma emoção natural e passageira. Ao lidar com ela de forma calma e consciente, você pode transformar situações inesperadas em oportunidades de aprend